In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from ansys.mapdl.core import launch_mapdl
import sys

# Launch MAPDL
mapdl = launch_mapdl(nproc=4) # Launch using 4 CPU Threads
mapdl.clear()

In [ ]:
def freq_name(no_freq, include_freq=True, include_ratio=True):
    """
    Creates an ordered list of string from inputted parameters:

    no_freq = (int) number of desired frequencies
    include_freq = (bool) include the individual frequencies or not (default True)
    include_ratio = (bool) include the non-trivial ratios between frequencies or not (default True)
    """
    names = []
    if include_freq:
        for i in range(no_freq):
            names.append('f'+str(i+1))
    if include_ratio:
        for i in range(no_freq):
            for j in range(i):
                names.append('f'+str(i+1)+'/f'+str(j+1))
    return names

In [ ]:
# Parametric Model as a Python Function
def modal_plate(plate_geom_mat_array, N_modes=16, N_div_b=10, EFS=0):
    """
    Run modal analysis on a rectangular plate
    
    Inputs:
    --------
    plate_geom_mat_array = np.array([ E, nu, rho, a, b, t ])
    N_modes = integer number of deformation modes to analyse (default=10)
    N_div_b = integer number of divisions for plate width (default=10)

    where:
    E = Elastic Modulus [Pa]
    nu = Poisson's ratio
    rho = Density [kg/m^3]
    a = Length [m]
    b = Width [m]
    t = Thickness [m]

    Outputs:
    --------
    PyMAPDL Solution Object

    """
    # Initial Set Up
    mapdl.clear() # Clear any previous PyMAPDL instances
    mapdl.prep7() # Start prepartion module
    mapdl.units("SI")  # Specify SI as unit system

    # Access geometric and material parameters
    E,nu,rho,a,b,t = plate_geom_mat_array

    # Ensure a>b
    if b>a:
        a, b = b, a

    # Calculates the number of divisions for plate length
    N_div_a = np.round(a/b*N_div_b)
    
    if N_div_a%2==1:
        N_div_a += 1

    # Defines a shell element 
    mapdl.et(1,"Shell281") # Shell281 element instance created
    mapdl.sectype(1, "SHELL")
    mapdl.secdata(t)

    # Defines a material model (SI)
    mapdl.mp("EX", 1, E)  # Elastic modulus in Pa 
    mapdl.mp("DENS", 1, rho)  # Density in kg/m3
    mapdl.mp("NUXY", 1, nu)  # Poisson's ratio
    mapdl.emodif("ALL", "MAT", 1) # Apply material model to all elements

    # Defines the geometry
    plate_A = mapdl.blc4(width=b, height=a) # Create rectangular section

    # Meshing
    mapdl.aclear("all") # Clear any selections
    mapdl.lesize("1",ndiv=N_div_b) # Specify number of divisions along plate width (bottom)
    mapdl.lesize("2",ndiv=N_div_a) # Specify number of divisions along plate length (right)
    mapdl.lesize("3",ndiv=N_div_b) # Specify number of divisions along plate width (top)
    mapdl.lesize("4",ndiv=N_div_a) # Specify number of divisions along plate length (left)
    mapdl.amesh(plate_A) # Mesh Geometry

    mapdl.et(2, "SURF154", kop4=0)
    mapdl.r(2,r4=EFS)
 
    # mapdl.nsel("s", "loc", "z", -0.001, 0.001)
    mapdl.type(2)
    mapdl.real(2)
    
    mapdl.esurf()
    
    # Solve the Eigenvalue Problem
    mapdl.modal_analysis(nmode=N_modes, freqb=0) # Analyse desired number of modes and filter out rigid modes
    result = mapdl.result # Store PyMAPDL Solution Object
    return result 

In [ ]:
def plotmodeshapes(mapdlresult, save=False, savename='ModeShapes.pdf'):
    coord = mapdlresult.mesh.nodes
    nnum, sol1 = mapdlresult.nodal_solution(6+0)
    nnum, sol2 = mapdlresult.nodal_solution(6+1)
    nnum, sol3 = mapdlresult.nodal_solution(6+2)
    nnum, sol4 = mapdlresult.nodal_solution(6+3)
    b, a, t = coord.max(axis=0)*1000

    fig, axs = plt.subplots(2, 2, figsize=(7, 7))
    fig.tight_layout(pad=5)

    plot1 = axs[0, 0].tricontourf(coord[:,0]*1000, coord[:,1]*1000, sol1[:,2]/np.max(np.abs(sol1[:,2])), cmap='coolwarm')
    axs[0, 0].set_title('Mode Shape 1')
    axs[0, 0].set(xlabel=r'$x$ (Length) [mm]', ylabel=r'$y$ (Width) [mm]')
    axs[0, 0].set_xlim(0,b)
    axs[0, 0].set_ylim(0,a)

    plot2 =axs[0, 1].tricontourf(coord[:,0]*1000, coord[:,1]*1000, sol2[:,2]/np.max(np.abs(sol2[:,2])), cmap='coolwarm')
    axs[0, 1].set_title('Mode Shape 2')
    axs[0, 1].set(xlabel=r'$x$ (Length) [mm]', ylabel=r'$y$ (Width) [mm]')
    axs[0, 1].set_xlim(0,b)
    axs[0, 1].set_ylim(0,a)

    plot3 =axs[1, 0].tricontourf(coord[:,0]*1000, coord[:,1]*1000, sol3[:,2]/np.max(np.abs(sol3[:,2])), cmap='coolwarm')
    axs[1, 0].set_title('Mode Shape 3')
    axs[1, 0].set(xlabel=r'$x$ (Length) [mm]', ylabel=r'$y$ (Width) [mm]')
    axs[1, 0].set_xlim(0,b)
    axs[1, 0].set_ylim(0,a)
    
    plot4 =axs[1, 1].tricontourf(coord[:,0]*1000, coord[:,1]*1000, sol4[:,2]/np.max(np.abs(sol4[:,2])), cmap='coolwarm')
    axs[1, 1].set_title('Mode Shape 4')
    axs[1, 1].set(xlabel=r'$x$ (Length) [mm]', ylabel=r'$y$ (Width) [mm]')
    axs[1, 1].set_xlim(0,b)
    axs[1, 1].set_ylim(0,a)

    fig.colorbar(plot1, ax=axs[0, 0])
    fig.colorbar(plot2, ax=axs[0, 1])
    fig.colorbar(plot3, ax=axs[1, 0])
    fig.colorbar(plot4, ax=axs[1, 1])
    if save:
        fig.savefig(savename, dpi=1200, bbox_inches='tight')

In [ ]:
nu, a, b, t = 0.2, 500/1000, 125/1000, 50/1000

E, rho = 200e9, 8000
c = np.sqrt(E/rho)

factors = np.linspace(1, 1.2, 20)
E_vals = []
rho_vals = []
freqs = []

for i in factors:
    nu_new = i*(1+nu)-1
    E_vals.append(nu_new)
    rho_vals.append(rho/i)
    freqs.append(modal_plate(np.array([E,nu_new,i*rho,a,b,t]), 16, 10, 1e7).time_values[6:])

freqs = np.array(freqs)
E_vals = np.array(E_vals)
rho_vals = np.array(rho_vals)

plt.figure(figsize=(5,4))
plt.title('Speed of Sound Study (c = {0:.4g} m/s)'.format(c))
plt.ylabel('Frequency [Hz]')
plt.xlabel('Elastic Modulus [GPa]')
# plt.ylim(0, 4000)
# plt.xlim(50, 350)
for i in range(10):
    plt.plot(E_vals/1e9, freqs[:,i], label='f{}'.format(i+1))
plt.legend(bbox_to_anchor=(1, 1))
plt.savefig('c_study_steel.pdf', dpi=1200, bbox_inches='tight')
plt.show()

In [ ]:
nu, a, b, t = 0.4, 500/1000, 125/1000, 50/1000

E, rho = 200e9, 8000
c = np.sqrt(E/rho)

factors = np.linspace(70e9/E, 300e9/E, 20)
E_vals = []
rho_vals = []
freqs = []

for i in factors:
    E_vals.append(i*E)
    rho_vals.append(i*rho)
    freqs.append(modal_plate(np.array([i*E,nu,i*rho,a,b,t]), 16, 10, 1e7).time_values[6:])

freqs = np.array(freqs)
E_vals = np.array(E_vals)
rho_vals = np.array(rho_vals)

plt.figure(figsize=(5,4))
plt.title('Speed of Sound Study (c = {0:.4g} m/s)'.format(c))
plt.ylabel('Frequency [Hz]')
plt.xlabel('Elastic Modulus [GPa]')
# plt.ylim(0, 4000)
# plt.xlim(50, 350)
for i in range(10):
    plt.plot(E_vals/1e9, freqs[:,i], label='f{}'.format(i+1))
plt.legend(bbox_to_anchor=(1, 1))
plt.savefig('c_study_steel.pdf', dpi=1200, bbox_inches='tight')
plt.show()

In [ ]:
nu, a, b, t = 0.3, 225/1000, 125/1000, 5/1000

E, rho = 130e9, 9000
c = np.sqrt(E/rho)

factors = np.linspace(70e9/E, 300e9/E, 100)
E_vals = []
rho_vals = []
freqs = []

for i in factors:
    E_vals.append(i*E)
    rho_vals.append(i*rho)
    freqs.append(modal_plate(np.array([i*E,nu,i*rho,a,b,t]), 16, 10, 1e7).time_values[6:])

freqs = np.array(freqs)
E_vals = np.array(E_vals)
rho_vals = np.array(rho_vals)

plt.figure(figsize=(5,4))
plt.title('Speed of Sound Study (c = {0:.4g} m/s)'.format(c))
plt.ylabel('Frequency [Hz]')
plt.xlabel('Elastic Modulus [GPa]')
plt.ylim(0, 3000)
plt.xlim(50, 350)
for i in range(10):
    plt.plot(E_vals/1e9, freqs[:,i], label='f{}'.format(i+1))
plt.legend(bbox_to_anchor=(1, 1))
plt.savefig('c_study_copper.pdf', dpi=1200, bbox_inches='tight')
plt.show()

In [ ]:
E, rho, a, b, t = 200e9, 8000, 400/1000, 200/1000, 10/1000

nu_vals = np.linspace(0.05, 0.45, 100+1)

freqs = []

for nu in nu_vals:
    freqs.append(modal_plate(np.array([E,nu,rho,a,b,t]), 16, 10, 1e7).time_values[6:])

freqs = np.array(freqs)

plt.figure(figsize=(5,4))
plt.title('Poisson Ratio Study (Steel)')
plt.ylabel('Frequency [Hz]')
plt.xlabel("Poisson's Ratio ")
plt.ylim(0, 2500)
plt.xlim(0, 0.5)
for i in range(10):
    plt.plot(nu_vals, freqs[:,i], label='f{}'.format(i+1))
plt.legend(bbox_to_anchor=(1, 1))
plt.savefig('nu_study_steel.pdf', dpi=1200, bbox_inches='tight')
plt.show()

nu_plate_steel1 = modal_plate(np.array([E,0.05,rho,a,b,t]), 16, 10, 1e7)
plotmodeshapes(nu_plate_steel1, 1, 'ModeShapes_nu1_steel.pdf')
nu_plate_steel2 = modal_plate(np.array([E,0.45,rho,a,b,t]), 16, 10, 1e7)
plotmodeshapes(nu_plate_steel2, 1, 'ModeShapes_nu2_steel.pdf')

In [ ]:
E, rho, a, b, t = 70e9, 2700, 300/1000, 180/1000, 4/1000

nu_vals = np.linspace(0.05, 0.45, 100+1)

freqs = []

for nu in nu_vals:
    freqs.append(modal_plate(np.array([E,nu,rho,a,b,t]), 16, 10, 1e7).time_values[6:])

freqs = np.array(freqs)

plt.figure(figsize=(5,4))
plt.title('Poisson Ratio Study (Aluminium)')
plt.ylabel('Frequency [Hz]')
plt.xlabel("Poisson's Ratio ")
plt.ylim(0, 1750)
plt.xlim(0, 0.5)
for i in range(10):
    plt.plot(nu_vals, freqs[:,i], label='f{}'.format(i+1))
plt.legend(bbox_to_anchor=(1, 1))
plt.savefig('nu_study_aluminium.pdf', dpi=1200, bbox_inches='tight')
plt.show()

nu_plate_aluminium1 = modal_plate(np.array([E,0.05,rho,a,b,t]), 16, 10, 1e7)
plotmodeshapes(nu_plate_aluminium1, 1, 'ModeShapes_nu1_aluminium.pdf')
nu_plate_aluminium2 = modal_plate(np.array([E,0.45,rho,a,b,t]), 16, 10, 1e7)
plotmodeshapes(nu_plate_aluminium2, 1, 'ModeShapes_nu2_aluminium.pdf')

In [ ]:
E, nu, rho, a, b = 200e9, 0.29, 8000, 200/1000, 150/1000

ah_vals = np.linspace(5, 100, 95*4+1)

freqs = []

for ah in ah_vals:
    freqs.append(modal_plate(np.array([E,nu,rho,a,b,a/ah]), 16, 10, 1e7).time_values[6:])

freqs = np.array(freqs)

plt.figure(figsize=(5,4))
plt.title('Thickness Ratio Study (Steel)')
plt.ylabel('Frequency [Hz]')
plt.xlabel(r"Thickness Ratio $a/h$")
plt.ylim(0, 14000)
plt.xlim(0, 105)
for i in range(10):
    plt.plot(ah_vals, freqs[:,i], label='f{}'.format(i+1))
plt.legend(bbox_to_anchor=(1, 1))
plt.savefig('ah_study_steel.pdf', dpi=1200, bbox_inches='tight')
plt.show()

In [ ]:
E, nu, rho, a, b = 70e9, 0.29, 2700, 400/1000, 200/1000

ah_vals = np.linspace(5, 100, 95*4+1)

freqs = []

for ah in ah_vals:
    freqs.append(modal_plate(np.array([E,nu,rho,a,b,a/ah]), 16, 10, 1e7).time_values[6:])

freqs = np.array(freqs)

plt.figure(figsize=(5,4))
plt.title('Thickness Ratio Study (Aluminium)')
plt.ylabel('Frequency [Hz]')
plt.xlabel(r"Thickness Ratio $a/h$")
plt.ylim(0, 9000)
plt.xlim(0, 105)
for i in range(10):
    plt.plot(ah_vals, freqs[:,i], label='f{}'.format(i+1))
plt.legend(bbox_to_anchor=(1, 1))
plt.savefig('ah_study_aluminium.pdf', dpi=1200, bbox_inches='tight')
plt.show()

In [ ]:
E, nu, rho, a_b, a_h = 200e9, 0.3, 8000, 1, 20

b_vals = np.linspace(50,500, 45+1)/1000

freqs = []

for b in b_vals:
    freqs.append(modal_plate(np.array([E,nu,rho,a_b*b,b,a_b*b/a_h]), 16, 10, 1e7).time_values[6:])

freqs = np.array(freqs)
plt.figure(figsize=(5,4))
plt.title('Thickness Ratio Study (Aluminium)')
plt.ylabel('Frequency [Hz]')
plt.xlabel(r"Thickness Ratio $a/h$")
# plt.ylim(0, 9000)
# plt.xlim(0, 105)
for i in range(10):
    plt.plot(b_vals, freqs[:,i], label='f{}'.format(i+1))
plt.legend(bbox_to_anchor=(1, 1))
plt.savefig('b_study_steel.pdf', dpi=1200, bbox_inches='tight')
plt.show()

In [ ]:
E, nu, rho, a_b, a_h = 70e9, 0.3, 2700, 2.5, 30

b_vals = np.linspace(50,500, 45+1)/1000

freqs = []

for b in b_vals:
    freqs.append(modal_plate(np.array([E,nu,rho,a_b*b,b,a_b*b/a_h]), 16, 10, 1e7).time_values[6:])

freqs = np.array(freqs)
plt.figure(figsize=(5,4))
plt.title('Thickness Ratio Study (Aluminium)')
plt.ylabel('Frequency [Hz]')
plt.xlabel(r"Thickness Ratio $a/h$")
# plt.ylim(0, 9000)
# plt.xlim(0, 105)
for i in range(10):
    plt.plot(b_vals, freqs[:,i], label='f{}'.format(i+1))
plt.legend(bbox_to_anchor=(1, 1))
plt.savefig('b_study_steel.pdf', dpi=1200, bbox_inches='tight')
plt.show()

In [ ]:
E, nu, rho, a_b, a_h = 135e9, 0.3, 9000, 1.75, 40

b_vals = np.linspace(50,1000, 95+1)/1000

freqs = []

for b in b_vals:
    freqs.append(modal_plate(np.array([E,nu,rho,a_b*b,b,a_b*b/a_h]), 16, 10, 1e7).time_values[6:])

freqs = np.array(freqs)
plt.figure(figsize=(5,4))
plt.title('Thickness Ratio Study (Aluminium)')
plt.ylabel('Frequency [Hz]')
plt.xlabel(r"Thickness Ratio $a/h$")
# plt.ylim(0, 9000)
# plt.xlim(0, 105)
for i in range(10):
    plt.semilogy(b_vals, freqs[:,i], label='f{}'.format(i+1))
plt.legend(bbox_to_anchor=(1, 1))
plt.savefig('b_study_steel.pdf', dpi=1200, bbox_inches='tight')
plt.show()

In [ ]:
rho, nu, a, b, t = 5000,0.3, 225/1000, 125/1000, 10/1000

factors = np.linspace(2000, 10000, 4*80+1)
freqs = []

for i in factors:
    freqs.append(modal_plate(np.array([i**2*rho,nu,rho,a,b,t]), 16, 10, 1e7).time_values[6:])

freqs = np.array(freqs)

plt.figure(figsize=(5,4))
plt.title('Speed of Sound Study')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Speed of Sound [m/s]')
# plt.ylim(0, 3000)
# plt.xlim(50, 350)
for i in range(10):
    plt.plot(factors, freqs[:,i], label='f{}'.format(i+1))
plt.legend(bbox_to_anchor=(1, 1))
plt.savefig('c_study.pdf', dpi=1200, bbox_inches='tight')
plt.show()

In [ ]:
import scipy.interpolate as scip
import time as time

E = 80e9
nu = 0.33
rho = 2700
a = 0.5
b = 0.25
t = 0.01


res = modal_plate(np.array([E,nu,rho,a,b,t]), 16, 10, 1e7)
coords = res.mesh.nodes
nnum, shape = res.nodal_solution(8)

coords = coords[:,0:2]
shape = np.abs(shape[:,2]/np.max(np.abs(shape[:,2])))

plt.figure(figsize=(6,6))
plt.tricontourf(coords[:,0], coords[:,1], shape, levels=20)
plt.colorbar()
plt.show()

In [ ]:
t1 = time.time()
a_interp = np.linspace(0, a, 8+1, endpoint=1)
b_interp = np.linspace(0, b, 8+1, endpoint=1)

B_interp, A_interp = np.meshgrid(b_interp, a_interp)

shape_interp = scip.griddata(coords, shape, (B_interp, A_interp), method='cubic')
t2 = time.time()

print(t2-t1)

plt.figure(figsize=(6,6))
plt.contourf(B_interp, A_interp, shape_interp, levels=20)
plt.scatter(B_interp, A_interp)
plt.colorbar()
plt.show()

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('Data Workspace/FM_TV_Data.csv')

df['E'].hist()

In [ ]:
df['rho'].hist()

In [ ]:
df['c'].hist()

In [ ]:
np.concatenate([np.arange(10), np.arange(5)])

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('Data Workspace\Fu_Te_Data.csv')

In [ ]:
x_axis = 'f1'
x_plot = np.log(df[x_axis])

fig, axs = plt.subplots(2, 2, figsize=(12,8))
axs[0, 0].scatter(x_plot, df['c'], marker='.', label='c')
axs[0, 0].legend()
axs[0, 1].scatter(x_plot, df['nu'], marker='.', label='nu')
axs[0, 1].legend()
axs[1, 0].scatter(x_plot, df['a/b'], marker='.', label='a/b')
axs[1, 0].legend()
axs[1, 1].scatter(x_plot, df['a/h'], marker='.', label='a/h')
axs[1, 1].legend()
plt.show()

In [ ]:
%matplotlib widget
plt.figure()
ax = plt.axes(projection='3d')
ax.scatter(df['f1'],df['f2'],df['f3'], c=df['a/b'], marker='.', s=1)
plt.show()

In [ ]:
plt.figure()
ax = plt.axes(projection='3d')
ax.scatter(df['f8/f7'],df['f9/f7'],df['f10/f7'], c=df['a/b'], marker='.', s=1)
plt.show()

plt.figure()
ax = plt.axes(projection='3d')
ax.scatter(df['f8/f6'],df['f9/f6'],df['f10/f6'], c=df['a/b'], marker='.', s=1)
plt.show()

plt.figure()
ax = plt.axes(projection='3d')
ax.scatter(df['f8/f5'],df['f9/f5'],df['f10/f5'], c=df['a/b'], marker='.', s=1)
plt.show()

plt.figure()
ax = plt.axes(projection='3d')
ax.scatter(df['f8/f4'],df['f9/f4'],df['f10/f4'], c=df['a/b'], marker='.', s=1)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
FS_TV = pd.read_csv('Data Workspace\Fu_TV_Data.csv')
RS_TV = pd.read_csv('Data Workspace\Re_TV_Data.csv')

In [ ]:
plt.figure(figsize=(6,4))
plt.title(r'One Dimensional Speed of Sound $c_1$ - FS Dataset')
plt.ylabel('Frequency')
plt.xlabel(r'$c_1$ [km/s]')
plt.ylim(0,5500)
plt.hist(FS_TV['c'], bins=20, rwidth=0.8, color='tab:green')
plt.savefig('FS_histogram_c.pdf', dpi=1200, bbox_inches='tight')
plt.show()

plt.figure(figsize=(6,4))
plt.title(r'One Dimensional Speed of Sound $c_1$ - RS Dataset')
plt.ylabel('Frequency')
plt.xlabel(r'$c_1$ [km/s]')
plt.ylim(0,2500)
plt.hist(RS_TV['c'], bins=20, rwidth=0.8, color='tab:green')
plt.savefig('RS_histogram_c.pdf', dpi=1200, bbox_inches='tight')
plt.show()

In [ ]:
import pandas as pd
import numpy as np

In [2]:
FS_TV = pd.read_csv('Data Workspace\FS_TV.csv')
RS_TV = pd.read_csv('Data Workspace\RS_TV.csv')
FS_Te = pd.read_csv('Data Workspace\FS_Te.csv')
RS_Te = pd.read_csv('Data Workspace\RS_Te.csv')

In [3]:
def freq_name(no_freq, include_freq=True, include_ratio=True):
    """
    Creates an ordered list of string from inputted parameters:

    no_freq = (int) number of desired frequencies
    include_freq = (bool) include the individual frequencies or not (default True)
    include_ratio = (bool) include the non-trivial ratios between frequencies or not (default True)
    """
    names = []
    if include_freq:
        for i in range(no_freq):
            names.append('f'+str(i+1))
    if include_ratio:
        for i in range(no_freq):
            for j in range(i):
                names.append('f'+str(i+1)+'/f'+str(j+1))
    return names

In [4]:
features = freq_name(10,1,1)
labels = ['E', 'nu', 'rho', 'a', 'b', 'h']

In [ ]:
expdf = pd.read_excel('Data Workspace/Experimental_Data.xlsx')

In [3]:
# expdf = expdf[features+labels]

expdf['G'] = expdf['E']/(2*(1+expdf['nu']))
expdf['K'] = expdf['E']/(3*(1-2*expdf['nu']))

expdf['ck'] = np.sqrt((expdf['K']+4/3*expdf['G'])/expdf['rho'])
expdf['cg'] = np.sqrt((expdf['G'])/expdf['rho'])
expdf['c1'] = np.sqrt((expdf['E'])/expdf['rho'])

expdf['a/b'] = expdf['a']/expdf['b']
expdf['a/h'] = expdf['a']/expdf['h']
expdf['b/h'] = expdf['b']/expdf['h']
expdf['V'] = expdf['a']*expdf['b']*expdf['h']
expdf['m'] = expdf['rho']*expdf['V']

expdf['D'] = expdf['E']*expdf['h']**3 / (12* (1-expdf['nu']**2))
expdf['Dk'] = expdf['E']*expdf['h'] / (12* (1-expdf['nu']**2))
expdf['Ds'] = expdf['G']*expdf['h']**3 / 12

expdf['Q1'] = np.sqrt(expdf['D']/(expdf['rho']*expdf['h']))
expdf['Q2'] = (1/expdf['a']**2)*np.sqrt(expdf['D']/(expdf['rho']*expdf['h']))
expdf['Q3'] = (1/expdf['b']**2)*np.sqrt(expdf['D']/(expdf['rho']*expdf['h']))
expdf['Q4'] = (1/expdf['a']**2 + 1/expdf['b']**2)*np.sqrt(expdf['D']/(expdf['rho']*expdf['h']))

expdf['M1'] = expdf['G']*expdf['h']
expdf['M2'] = expdf['G']*expdf['h']/expdf['D']
expdf['M3'] = expdf['D']*expdf['nu']
expdf['M4'] = expdf['D']*(1-expdf['nu'])

expdf.to_csv('Data Workspace/Experimental_Data.csv',index=False)

In [1]:
import pandas as pd
import numpy as np

FS_TV = pd.read_csv('Data Workspace\FS_TV.csv')
RS_TV = pd.read_csv('Data Workspace\RS_TV.csv')
FS_Te = pd.read_csv('Data Workspace\FS_Te.csv')
RS_Te = pd.read_csv('Data Workspace\RS_Te.csv')

In [5]:
RS_Te['c1/a'] = RS_Te['c1']/RS_Te['a']
RS_Te['c1/b'] = RS_Te['c1']/RS_Te['b']
RS_Te.to_csv('RS_Te.csv',index=False)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
FS_TV = pd.read_csv('Data Workspace\FS_TV.csv')

In [ ]:
%matplotlib widget
plt.figure()
ax = plt.axes(projection='3d')
ax.scatter(FS_TV['f1'],FS_TV['f2'],FS_TV['f3'], c=FS_TV['a/b'],s=1)
plt.show()

In [ ]:
FS_mape = [20.23718853,	13.90613333,	11.81406667,	7.258466667,	6.658,	5.328933333,	4.624066667,	4.207166667,	3.3407]
RS_mape = [27.11408485,	19.80346667,	18.3914,	11.08273333,	10.0172,	7.7926,	6.8625,	6.067566667,	4.671866667]
ES_mape = [51.10705748,	23.22620732,	20.35354258,	14.62001209,	12.92993543,	9.683623719,	8.29300237,	8.305190242,	6.68508846]
plt.figure(figsize=(5,5))
plt.title('Model Error vs Number of Frequencies')
plt.ylabel('MAPE [%]')
plt.xlabel(['Number of Frequencies'])
plt.plot(np.arange(2,11), FS_mape, 'o-', label='FS Test Set')
plt.plot(np.arange(2,11), RS_mape, 's:', label='RS Test Set')
plt.plot(np.arange(2,11), ES_mape, '^--', label='Sim Exp Plates')
plt.xticks(np.arange(1,12))
plt.ylim(0,55)
plt.grid()
plt.legend()
plt.savefig('MAPE_nFreq.pdf', dpi=1200, bbox_inches='tight')
plt.show()